In [178]:
is_train = True
basic_epochs = 0
local = 1
if (local):
    log_dir = '/home/zyp/w/d2l/classify_leave/log'
    data_dir = '/home/zyp/w/d2l/classify_leave/data'
    model_file = '/home/zyp/w/d2l/classify_leave/log/epochs'+str(basic_epochs)+'.model'
else:
    log_dir = '/content/log'
    data_dir = '/content/data'
    model_file = '/content/log/epochs'+str(basic_epochs)+'.model'
    drive_log_dir = '/content/drive/MyDrive/colab/classify_leave/log'
    drive_model_file = '/content/drive/MyDrive/colab/classify_leave/log/epochs'+str(basic_epochs)+'.model'
    
!mkdir -p $log_dir
!mkdir -p $data_dir

total_epochs = basic_epochs
log_num = 0

In [19]:
#显示运行时间
!pip install ipython-autotime
%load_ext autotime

# 挂载 drive
from google.colab import drive
drive.mount('/content/drive')

# 准备数据集

!cp /content/drive/MyDrive/colab/classify_leave/classify-leaves.zip /content/data
!unzip /content/data/classify-leaves.zip -d /content/data/
!ls /content/data/images/ | wc -l

#!cp $drive_model_file /content/log/


The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  /content/data/classify-leaves.zip
replace /content/data/images/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 27153
time: 23.4 s (started: 2022-05-20 07:24:05 +00:00)


In [179]:
import math
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import torchvision.models as models
from torchvision.io import read_image
import logging
import torchvision.transforms as transforms

In [184]:
logging.basicConfig(filename=log_dir + '/1.log', level=logging.DEBUG, format='%(message)s')
logging.info('abc')
log_file = log_dir + '/1.log'
print(log_file)
!cat $log_file

/home/zyp/w/d2l/classify_leave/log/1.log
abc
abc


In [185]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [186]:
all_train_label = pd.read_csv(data_dir + '/train.csv')
l = len(all_train_label)
train_set = all_train_label[0: math.floor(l/2)]
validation_set = all_train_label[math.floor(l/2):]

print(train_set)

                image                  label
0        images/0.jpg       maclura_pomifera
1        images/1.jpg       maclura_pomifera
2        images/2.jpg       maclura_pomifera
3        images/3.jpg       maclura_pomifera
4        images/4.jpg       maclura_pomifera
...               ...                    ...
9171  images/9171.jpg        aesculus_glabra
9172  images/9172.jpg           quercus_alba
9173  images/9173.jpg      cercis_canadensis
9174  images/9174.jpg  gleditsia_triacanthos
9175  images/9175.jpg       maclura_pomifera

[9176 rows x 2 columns]


In [187]:
leave_class = sorted(list(set(all_train_label['label'])))
class_to_idx = dict(zip(leave_class, range(len(leave_class)))) 
idx_to_class = {v:k for k, v in class_to_idx.items()}

print(list(class_to_idx.items())[0:4])
print(list(idx_to_class.items())[0:4])
print(class_to_idx['abies_nordmanniana'])

[('abies_concolor', 0), ('abies_nordmanniana', 1), ('acer_campestre', 2), ('acer_ginnala', 3)]
[(0, 'abies_concolor'), (1, 'abies_nordmanniana'), (2, 'acer_campestre'), (3, 'acer_ginnala')]
1


In [188]:
class LeaveDataset(Dataset):
    
    all_train_label = pd.read_csv(data_dir + '/train.csv')
    test_label = pd.read_csv(data_dir + '/test.csv')
    offset = math.floor(len(all_train_label) / 2)
    train_mode = "train"
    validation_mode = "validation"
    
    def __init__(self, mode):
        self.mode = mode        
        
    def __getitem__(self, idx):
        all_train_label = LeaveDataset.all_train_label.copy(deep=False)
        test_label = LeaveDataset.test_label.copy(deep=False)

        if self.mode == LeaveDataset.train_mode:
            img_file, label = all_train_label.iloc[idx][0], all_train_label.iloc[idx][1]
        elif self.mode == LeaveDataset.validation_mode:
            idx = LeaveDataset.offset + idx
            img_file, label = all_train_label.iloc[idx][0], all_train_label.iloc[idx][1]
        else:
            img_file = test_label.iloc[idx][0]
            
        img = read_image(str(data_dir + '/' + img_file))
        if self.mode == LeaveDataset.train_mode:
            t = transforms.Compose([
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.RandomResizedCrop(size=(224, 224), scale=(0.5, 1)),
                transforms.ConvertImageDtype(torch.float),
            ])
        else:
            t = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ConvertImageDtype(torch.float),
            ])         
        
        img = t(img)
        if 'label' in locals():
            label = class_to_idx[label]
            return img, label
        else:
            return img

    def __len__(self):
        test_label = LeaveDataset.test_label.copy(deep=False)

        if self.mode == LeaveDataset.train_mode:
            return LeaveDataset.offset
        elif self.mode == LeaveDataset.validation_mode:
            return len(all_train_label) - LeaveDataset.offset 
        else:
            return len(test_label)

        
training_data = LeaveDataset("train")
validation_data = LeaveDataset("validation")
test_data = LeaveDataset("test")
print(len(training_data), len(validation_data), len(test_data))

train_dataloader = DataLoader(training_data, batch_size=64)
validation_dataloader = DataLoader(validation_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


9176 9177 8800


In [190]:
model = models.resnet50(pretrained=True)

#model = models.resnet50(pretrained=False)
#model.load_state_dict(torch.load(model_file, map_location=torch.device(device)))

model.train(is_train)
model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/zyp/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [191]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# Initialize the loss function / optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [192]:
for batch, (X, y) in enumerate(train_dataloader):
    X, y = X.to(device), y.to(device)
    pred = model(X)
    loss = loss_fn(pred, y)
    print(y)
    print(pred.argmax(1))
    print(idx_to_class[78])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    break



tensor([ 78,  78,  78,  78,  78,  78, 174,  28,  78,  28,  78, 130,  78, 130,
         28,  78,  78,  28,  78, 174,  78,  78, 130,  28,  78,  28, 174, 130,
         78, 174, 130,  10,  28,  78,  58,  78,  78,  10,  78,  10, 174,  78,
         10,  10,  58,  10,  28, 130, 130,  28, 130, 130,  28,  10, 130,  28,
         78,  78,  28, 174,  10,  58,  28, 130])
tensor([852, 879, 588, 758, 512, 790, 749, 356, 700, 426, 859, 980, 956, 608,
        788, 641, 515, 793, 859, 329, 373, 943, 827, 530, 488, 455, 748, 844,
        859, 421, 749,  49,  45,  36, 457, 818, 891, 968, 577, 304, 868, 391,
        600, 311, 616, 532, 494, 448, 608, 431, 124,  14, 784, 745,  78, 749,
        475, 494, 611, 551, 428, 566, 626, 469])
maclura_pomifera


In [193]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    correct = 0
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            log = f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]"
            logging.info(log)
            print(log)
        
    correct /= size
    log = f"[Train] Accuracy: {(100*correct):>0.1f}% \n"
    print(log)
    logging.info(log)


def validation_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    log = f"[Test] Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    print(log)
    logging.info(log)


In [196]:
train_loop(train_dataloader, model, loss_fn, optimizer)
#validation_loop(validation_dataloader, model, loss_fn)

loss: 5.752347  [    0/ 9176]
loss: 5.846805  [ 6400/ 9176]
[Train] Accuracy: 5.9% 



In [33]:
epochs = 20
for t in range(epochs):
    total_epochs+=1
    log = f"Epoch {total_epochs}\n-------------------------------"
    print(log)
    logging.info(log)
    train_loop(train_dataloader, model, loss_fn, optimizer)
    validation_loop(validation_dataloader, model, loss_fn)
print("Done!")

print("save model at epochs " + str(total_epochs))
torch.save(model.state_dict(), log_dir+'/epochs' + str(total_epochs) + '.model')

!ls -alh $log_dir


Epoch 101
-------------------------------
loss: 0.444763  [    0/ 9176]
loss: 0.336020  [ 6400/ 9176]
[Train] Accuracy: 91.6% 

[Test] Accuracy: 79.8%, Avg loss: 0.733492 

Epoch 102
-------------------------------
loss: 0.433996  [    0/ 9176]
loss: 0.292215  [ 6400/ 9176]
[Train] Accuracy: 91.7% 

[Test] Accuracy: 80.1%, Avg loss: 0.729760 

Epoch 103
-------------------------------
loss: 0.414370  [    0/ 9176]
loss: 0.349467  [ 6400/ 9176]
[Train] Accuracy: 92.0% 

[Test] Accuracy: 80.5%, Avg loss: 0.718748 

Epoch 104
-------------------------------
loss: 0.288408  [    0/ 9176]
loss: 0.238979  [ 6400/ 9176]
[Train] Accuracy: 92.1% 

[Test] Accuracy: 80.2%, Avg loss: 0.719458 

Epoch 105
-------------------------------
loss: 0.395620  [    0/ 9176]
loss: 0.350331  [ 6400/ 9176]
[Train] Accuracy: 92.5% 

[Test] Accuracy: 80.2%, Avg loss: 0.719019 

Epoch 106
-------------------------------
loss: 0.445504  [    0/ 9176]
loss: 0.302794  [ 6400/ 9176]
[Train] Accuracy: 92.5% 

[Test] 

In [34]:
!mv /content/log/*.model $drive_log_dir
drive_log_file = drive_log_dir + "/log.epochs" + str(basic_epochs) + "." + str(log_num)
!cp /content/log/1.log $drive_log_file

time: 751 ms (started: 2022-05-20 08:06:08 +00:00)


---
## 测试


In [ ]:
# 测试集预测
test_pred = torch.tensor([], dtype=torch.int64)

with torch.no_grad():
    for X in test_dataloader:
        X = X.to(device)
        pred = model(X)
        test_pred = torch.hstack((test_pred, pred.argmax(1)))
        
print(test_pred.shape)

test_class = []
for i in enumerate(test_pred.numpy()):
    test_class.append(idx_to_class[i[1]])
print(len(test_class))
test_class2 = pd.DataFrame(test_class, columns=['label'])

result = pd.concat([LeaveDataset.test_label, test_class2], axis=1)
result.to_csv(log_dir + "/result.csv", index=False)
result